# Автоматизация отчётности 

##### Задачи
1. Собрать единый отчет по работе всего приложения. В отчете должна быть информация и по ленте новостей, и по сервису отправки сообщений. 
2. Продумать, какие метрики необходимо отобразить в этом отчете? Как можно показать их динамику? Приложить к отчету графики или файлы, чтобы сделать его более наглядным и информативным. 
3. Автоматизировать отправку отчета с помощью GitLab CI/CD. Отчет должен приходить ежедневно в 11:00 в чат. 

In [13]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io  #позволяет направлять потоки ввода вывода данных
import logging
import pandas as pd
import pandahouse
import os
import datetime as dt
#from read_db.CH import Getch


sns.set()

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220520'}


def test_report(chat=None):
    chat_id = chat or 726885196 #Чат ID личного чат бота
    bot = telegram.Bot(token='5388356743:AAFRvvBIu_oe-DprC7UJNiTajKpFf72QSnE')

    #Смотрим пользователей в ленте и мессенджере вместе за прошлую неделю

    q_all = '''SELECT 
                    toStartOfDay(toDateTime(time)) AS date, 
                    count(DISTINCT user_id) AS "Уникальные пользователи" 
                FROM (SELECT user_id, time FROM simulator_20220520.feed_actions t1 JOIN 
                      simulator_20220520.message_actions t2 on t1.user_id = t2.user_id) AS virtual_table
                WHERE date BETWEEN today()-7 AND today()
                GROUP BY toStartOfDay(toDateTime(time))
                ORDER BY date DESC'''

    #Смотрим пользователей только в ленте за прошлую неделю

    q_only_feed = '''SELECT toStartOfDay(toDateTime(time_action)) AS date, 
                            count(user_id) AS \"Уникальные пользователи\" 
                     FROM 
                           (SELECT * FROM (SELECT user_id, toDate(time) as time_action FROM simulator_20220520.feed_actions GROUP BY time_action, user_id) as t1 
                    LEFT JOIN 
                           (SELECT user_id, toDate(time) as time_message FROM simulator_20220520.message_actions GROUP BY time_message, user_id) as t2 on (t1.user_id=t2.user_id and t1.time_action=t2.time_message) 
                    WHERE time_action!=t2.time_message AND time_action 
                    BETWEEN today()-7 AND today() 
                    ORDER BY t1.user_id) AS virtual_table 
                    GROUP BY toStartOfDay(toDateTime(time_action))
                    ORDER BY date DESC'''

    #Смотрим пользователей только в мессенджере за прошлую неделю

    q_only_message = '''SELECT toStartOfDay(toDateTime(time_message)) AS date, 
                               count(user_id) AS "Уникальные пользователи" 
                        FROM
                            (SELECT * FROM (SELECT user_id, toDate(time) as time_message FROM simulator_20220520.message_actions GROUP BY time_message, user_id) as t1 
                        LEFT JOIN
                            (SELECT user_id, toDate(time) as time_action FROM simulator_20220520.feed_actions GROUP BY time_action, user_id) as t2 on (t1.user_id=t2.user_id and t1.time_message=t2.time_action) 
                        WHERE t1.time_message!=t2.time_action 
                        ORDER BY t1.user_id) AS virtual_table 
                        WHERE time_message 
                        BETWEEN today()-7 AND today() 
                        GROUP BY toStartOfDay(toDateTime(time_message)) 
                        ORDER BY date DESC'''

    #Смотрим общее количество просмотров, лайков и сообщений

    metrics_all = '''SELECT toStartOfDay(toDateTime(time)) AS date, 
                            countIf(action = 'like') AS "Лайки",
                            countIf(action = 'message') AS "Сообщения", 
                            countIf(action = 'view') AS "Просмотры" 
                    FROM
                            (SELECT user_id, gender,time, age, country, os, 'message' as action FROM simulator_20220520.message_actions
                    UNION ALL 
                            SELECT user_id, gender, time, age, country, os, action FROM simulator_20220520.feed_actions) AS virtual_table
                    WHERE date BETWEEN today()-7 AND today()
                    GROUP BY toStartOfDay(toDateTime(time))
                    ORDER BY date DESC'''
    
    #Создаём функцию для чтения датафреймов из sql
    def read_df(df):
        return pandahouse.read_clickhouse(df, connection=connection)
    
    time = dt.date.today() - dt.timedelta(days = 1)
    q_all_df = read_df(q_all)
    q_only_feed_df = read_df(q_only_feed)
    q_only_message_df = read_df(q_only_message)
    metrics_all_df = read_df(metrics_all)
    metrics_all_df['CTR'] = round(metrics_all_df.Лайки / metrics_all_df.Просмотры,2)

    list_dau = [q_all_df['Уникальные пользователи'][1], q_only_feed_df['Уникальные пользователи'][1], q_only_message_df['Уникальные пользователи'][1]]

    #Кладём в переменную msg текст, который будет отправлять наш бот

    msg = (f'Current data report: {time}') + '\n'\
          + (f'DAU message and feed: {list_dau[0]}') + '\n'\
          + (f'DAU only feed: {list_dau[1]}') + '\n'\
          + (f'Dau only message: {list_dau[2]}')

    bot.sendMessage(chat_id=chat_id, text=msg)

    #Строим графики по нашим запросам

    plt.figure(figsize = (20,15))

    plt.subplot(3,1,1)
    sns.lineplot(x = 'date', y = 'Уникальные пользователи', data = q_all_df)
    plt.subplot(3,1,1)
    sns.lineplot(x = 'date', y = 'Уникальные пользователи', data = q_only_feed_df)
    plt.subplot(3,1,1)
    sns.lineplot(x = 'date', y = 'Уникальные пользователи', data = q_only_message_df)
    plt.legend(labels = ['Пользутся мессенджером и новостной лентой', 'Пользуются только новостной лентой', 'Пользуются только мессенджером'])

    plt.subplot(3,1,2)
    sns.lineplot(x = 'date', y = 'Лайки', data = metrics_all_df)
    plt.subplot(3,1,2)
    sns.lineplot(x = 'date', y = 'Просмотры', data = metrics_all_df)
    plt.subplot(3,1,2)
    sns.lineplot(x = 'date', y = 'Сообщения', data = metrics_all_df)
    plt.legend(labels = ['Лайки', 'Просмотры', 'Сообщения'])

    plt.subplot(3,1,3)
    sns.lineplot(x = 'date', y = 'CTR', data = metrics_all_df)
    
    #Сохраняем наш график как файловый объет в буфере, чтобы потом была возможность доставать объет из буфера 
    
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.name = 'ALL_plot.png'
    plot_object.seek(0)

    plt.close()

    bot.sendPhoto(chat_id = chat_id, photo = plot_object)
       
try:
    test_report(-715060805)
except Exception as e:
    print(e)
